<a href="https://www.kaggle.com/code/masoudnaghshbandi/titanic-machine-learning-from-disaster?scriptVersionId=107948135" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [124]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


A bout data set, it has 11 columns,and most of them is clear what they are just a bit information about Embark which implies where the traveler mounted from. There are three possible values for Embark — Southampton, Cherbourg, and Queenstown.
parch: The dataset defines family relations in this way
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

# Import Libraries

In [125]:
from sklearn.model_selection import  train_test_split
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

# Load train and test data get into it

In [126]:
train_df=pd.read_csv('../input/titanic/train.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [127]:
train_df.shape

(891, 12)

In [128]:
test_df=pd.read_csv('../input/titanic/test.csv')
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [129]:
test_df.shape

(418, 11)

 ### In my idea passengerid, name, Ticket are not important so I will delet all of them 

In [130]:
train_df.drop(['PassengerId', 'Name','Ticket'], axis=1,inplace=True)
test_df.drop(['PassengerId', 'Name','Ticket'], axis=1,inplace=True)



## Deal with missing value

In [131]:
train_df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [132]:
train_df.drop(['Cabin'], axis=1,inplace=True)

Because age is a nature thing we can fill that with mean and embarked with mode.

In [133]:
train_df['Age'].fillna((train_df['Age'].mean()), inplace=True)


train_df['Embarked'].fillna( method ='ffill', inplace = True)


In [134]:
train_df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

We calculate the cabib mising value rate

In [135]:
train_df.isnull().sum() / train_df.shape[0] * 100.00


Survived    0.0
Pclass      0.0
Sex         0.0
Age         0.0
SibSp       0.0
Parch       0.0
Fare        0.0
Embarked    0.0
dtype: float64

Over 50% So it is better to eliminate this column too.

If we do the same for our Test data set:

In [136]:
test_df.isnull().sum()

Pclass        0
Sex           0
Age          86
SibSp         0
Parch         0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [137]:
test_df.isnull().sum() / train_df.shape[0] * 100.00


Pclass       0.000000
Sex          0.000000
Age          9.652076
SibSp        0.000000
Parch        0.000000
Fare         0.112233
Cabin       36.700337
Embarked     0.000000
dtype: float64

In [138]:
test_df['Age'].fillna((test_df['Age'].mean()), inplace=True)

test_df['Embarked'].fillna((test_df['Embarked'].mode()), inplace=True)

test_df['Fare'].fillna((test_df['Fare'].mean()), inplace=True)


In [139]:
test_df.drop(['Cabin'], axis=1,inplace=True)

In [140]:
test_df.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [141]:
train_df['Sex'].replace( 'female',1,inplace=True)
train_df['Sex'].replace( 'male',2,inplace=True)

In [142]:
test_df['Sex'].replace( 'female',1,inplace=True)
test_df['Sex'].replace('male',2,inplace=True)

In [143]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,2,22.0,1,0,7.2500,S
1,1,1,1,38.0,1,0,71.2833,C
2,1,3,1,26.0,0,0,7.9250,S
3,1,1,1,35.0,1,0,53.1000,S
4,0,3,2,35.0,0,0,8.0500,S


In [144]:
train_df['Embarked'].unique()

array(['S', 'C', 'Q'], dtype=object)

In [145]:
train_df['Embarked'].replace( 'S',1,inplace=True)
train_df['Embarked'].replace( 'C',2,inplace=True)
train_df['Embarked'].replace( 'Q',3,inplace=True)

In [146]:
test_df['Embarked'].replace( 'S',1,inplace=True)
test_df['Embarked'].replace( 'C',2,inplace=True)
test_df['Embarked'].replace( 'Q',3,inplace=True)

In [147]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,2,22.0,1,0,7.2500,1
1,1,1,1,38.0,1,0,71.2833,2
2,1,3,1,26.0,0,0,7.9250,1
3,1,1,1,35.0,1,0,53.1000,1
4,0,3,2,35.0,0,0,8.0500,1


In [148]:
X, y = train_df.drop('Survived', axis = 1), train_df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [149]:
scaler = StandardScaler()

In [150]:
scaler.fit_transform(X_train,X_test)

array([[ 0.84105013, -1.39360997, -0.20019692, ...,  2.10139559,
        -0.41607509, -0.56541298],
       [-0.36626377,  0.71756088,  1.69451327, ..., -0.45620041,
        -0.38850204, -0.56541298],
       [-0.36626377,  0.71756088, -0.20019692, ..., -0.45620041,
        -0.37909111, -0.56541298],
       ...,
       [-1.57357766,  0.71756088,  0.5103194 , ...,  2.10139559,
         1.74084582, -0.56541298],
       [ 0.84105013,  0.71756088,  0.01288832, ..., -0.45620041,
        -0.48261139, -0.56541298],
       [ 0.84105013,  0.71756088,  0.39190001, ..., -0.45620041,
        -0.5091106 ,  1.01330875]])

In [151]:
scaler.fit_transform(test_df)

array([[ 0.87348191,  0.75592895,  0.3349926 , ..., -0.4002477 ,
        -0.49840706,  2.24316522],
       [ 0.87348191, -1.32287566,  1.32553003, ..., -0.4002477 ,
        -0.51327429, -0.6778412 ],
       [-0.31581919,  0.75592895,  2.51417495, ..., -0.4002477 ,
        -0.46508846,  2.24316522],
       ...,
       [ 0.87348191,  0.75592895,  0.65196458, ..., -0.4002477 ,
        -0.50879189, -0.6778412 ],
       [ 0.87348191,  0.75592895,  0.        , ..., -0.4002477 ,
        -0.4944482 , -0.6778412 ],
       [ 0.87348191,  0.75592895,  0.        , ...,  0.61989583,
        -0.23790598,  0.78266201]])

In [152]:
#from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


In [153]:
#space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
       # 'gamma': hp.uniform ('gamma', 1,9),
       # 'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
      #  'reg_lambda' : hp.uniform('reg_lambda', 0,1),
       # 'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
      #  'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
       # 'n_estimators': 180,
   #     'seed': 0
    #}

In [154]:
#def objective(space):
   # clf=xgb.XGBClassifier(
                  #  n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                  #  reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                 #   colsample_bytree=int(space['colsample_bytree']))
    
   # evaluation = [( X_train, y_train), ( X_test, y_test)]
    
   # clf.fit(X_train, y_train,
            #eval_set=evaluation, eval_metric="auc",
          #  early_stopping_rounds=10,verbose=False)
    

    #pred = clf.predict(X_test)
   # accuracy = accuracy_score(y_test, pred>0.5)
  #  print ("SCORE:", accuracy)
  #  return {'loss': -accuracy, 'status': STATUS_OK }

In [155]:
#trials = Trials()

#best_hyperparams = fmin(fn = objective,
                   #     space = space,
                    #    algo = tpe.suggest,
                     #   max_evals = 100,
                      #  trials = trials)

In [156]:
#print("The best hyperparameters are : ","\n")
#print(best_hyperparams)

In [157]:
#model =xgb. XGBClassifier(n_estimators=2000, colsample_bytree= 0.782912138847328,gamma= 1.2484661685986642,max_depth=15,
                     min_child_weight=3.0, reg_alpha= 43.0,reg_lambda= 0.32889272247033874)
                     


IndentationError: unexpected indent (2836015574.py, line 2)

In [ ]:

#model.fit(X_train, y_train)

In [158]:
#expected_y  = y_test
#predicted_y = model.predict(X_test)

In [159]:
#print(metrics.classification_report(expected_y, predicted_y))
#print(metrics.confusion_matrix(expected_y, predicted_y))

In [160]:
#new_pred = model.predict(test_df)

In [161]:
#submission=pd.read_csv("../input/titanic/test.csv")

In [162]:
#submission['Survived'] = new_pred

In [163]:
#submission.head()

In [164]:
#submission.drop(['Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked'], axis=1,inplace=True)

In [165]:
#submission=submission.to_csv("submission.csv", index=False)

In [166]:
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [167]:
from sklearn.ensemble import RandomForestClassifier

In [183]:
rfc = RandomForestClassifier()


In [169]:
#from sklearn.model_selection import train_test_split, GridSearchCV

In [170]:
#grid_space={'max_depth':[3,5,10,None],
             # 'n_estimators':[10,100,200],
             # 'max_features':[1,3,5,7],
             # 'min_samples_leaf':[1,2,3],
             # 'min_samples_split':[1,2,3]
          # }

In [171]:
#grid = GridSearchCV(rfc,param_grid=grid_space,cv=3,scoring='accuracy')
#model_grid = grid.fit(X_train,y_train)

In [172]:
#print('Best hyperparameters are: '+str(model_grid.best_params_))
#print('Best score is: '+str(model_grid.best_score_))

In [184]:
rfc = RandomForestClassifier(max_depth=None, max_features=3,min_samples_leaf=3, min_samples_split=2,n_estimators= 100 )


In [185]:
model = rfc.fit(X_train,y_train)

In [186]:
expected_y  = y_test
predicted_y = model.predict(X_test)

In [187]:
print(metrics.classification_report(expected_y, predicted_y))
print(metrics.confusion_matrix(expected_y, predicted_y))

              precision    recall  f1-score   support

           0       0.75      0.90      0.82       100
           1       0.83      0.62      0.71        79

    accuracy                           0.78       179
   macro avg       0.79      0.76      0.76       179
weighted avg       0.79      0.78      0.77       179

[[90 10]
 [30 49]]


In [188]:
new_pred = model.predict(test_df)

In [189]:
submission=pd.read_csv("../input/titanic/test.csv")

In [190]:
submission['Survived'] = new_pred

In [191]:
submission.drop(['Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked'], axis=1,inplace=True)

In [192]:
submission=submission.to_csv("submissionnew.csv", index=False)